In [ ]:
%matplotlib widget
import numpy as np
from scipy.fft import fft, ifft
import matplotlib.pyplot as plt
from Functions.signal_generator import generate_ADSB, digitize_signal, correlate_signals, \
                                       normalize_signal, signal_start_pause_length, filter_with_window
from preambules_list import Preambule, preambule_list
from Functions.utils import SAMPLE_ADSB_BYTES

class ReverseCorrSampleBase:
    def __init__(self, correlation : np.ndarray, name):
        self.correlation = correlation
        self.result_mask = np.zeros(len(correlation))
        self.name = name

    def get_name(self) -> str:
        return self.name

    # Does deconvolution
    def do_reverse_correlation(self, signal: np.ndarray) -> np.ndarray:
        if len(self.correlation) != len(signal):
            raise ValueError(f"Correlation and signal must have the same length: {len(self.correlation)} != {len(signal)}")
        R_f = fft(self.correlation)
        X_f = fft(signal, len(self.correlation))

        # Regularization to avoid division by zero
        eps = 1e-10
        Y_f = R_f / (X_f + eps)

        self.result_mask = np.real(ifft(Y_f))

        # Remove outlying peaks from result_mask
        threshold = np.mean(self.result_mask) + 3 * np.std(self.result_mask)
        self.result_mask[self.result_mask > threshold] = threshold
        self.result_mask[self.result_mask < -threshold] = -threshold

    def display_reverse_corr(self, signal: np.ndarray):
        plt.figure()
        plt.suptitle(self.name)
        plt.subplot(311)
        plt.title("Signal")
        plt.stairs(signal)
        plt.ylabel("Amplitude")
        plt.subplot(312)
        plt.title("Correlation")
        plt.stairs(self.correlation)
        plt.ylabel("Amplitude")
        plt.subplot(313)
        plt.title("Mask")
        plt.stairs(self.result_mask)
        plt.ylabel("Amplitude")
        plt.xlabel("Sample, n")
        plt.tight_layout()
        plt.show()

example_correlation = np.array([0, 0, 0, 0, 1, 2, 3, 2, 1])
example_signal = np.array([0, 0, 0, 1, 1, 1, 0, 0, 0])
reverse_corr_example = ReverseCorrSampleBase(example_correlation, "Example")
reverse_corr_example.do_reverse_correlation(example_signal)
reverse_corr_example.display_reverse_corr(example_signal)

In [ ]:
def cutout_signals(signal: np.ndarray, preamble: np.ndarray):
    signal = signal[:1100]

    signal_corr = correlate_signals(signal, np.repeat(preamble.get_coefficients(), 1))

    CUTOFF_REGION_LOW = 0
    CUTOFF_REGION_HIGH = 300
    display_signal = signal[CUTOFF_REGION_LOW:CUTOFF_REGION_HIGH]
    signal_corr = signal_corr[CUTOFF_REGION_LOW:CUTOFF_REGION_HIGH]
    return display_signal, signal_corr

#Configuration
amplitude = 1
preamble = preambule_list[Preambule.Ideal.value]
expected_maximum = signal_start_pause_length + len(preamble.get_coefficients())
#Generate ADS-B signal
ideal_signal, filtered_signal, noisy_signal = generate_ADSB(amplitude, SAMPLE_ADSB_BYTES, generate_bits=False)
signal, _ = digitize_signal(noisy_signal, 100E6, 10E6, 1.4, 2**10)
# signal = noisy_signal

# Assemble needed signals
cutoff_normal_signal, cutoff_normal_corr = cutout_signals(signal, preamble)

impulse_corr = np.concatenate((np.zeros(expected_maximum), np.ones(1)))
impulse_corr = np.concatenate((impulse_corr, np.zeros(len(cutoff_normal_signal) - len(impulse_corr))))

sinc_x = np.linspace(-expected_maximum, len(cutoff_normal_signal) - expected_maximum, len(cutoff_normal_signal))
sinc_corr = np.sinc(sinc_x / np.pi)
sinc_corr = sinc_corr / np.max(np.abs(sinc_corr)) * np.max(np.abs(cutoff_normal_signal))

hamming_corr, centered_hamming_window = filter_with_window(cutoff_normal_corr, 100, expected_maximum)

plt.figure()
plt.plot(normalize_signal(cutoff_normal_corr), label = "Normal")
plt.plot(normalize_signal(centered_hamming_window), label = "Hamming")
plt.plot(normalize_signal(hamming_corr), label = "With hamming")
plt.legend()
plt.show()

#Insert signals into appropriate arrays for execution
reverse_corr_samples = [
    ReverseCorrSampleBase(cutoff_normal_corr, "Normal"),
    ReverseCorrSampleBase(hamming_corr, "With hamming"),
    ReverseCorrSampleBase(impulse_corr, "Impulse"),
    ReverseCorrSampleBase(sinc_corr, "Sinc"),
]

reverse_corr_signals = [
    [cutoff_normal_signal],
    [cutoff_normal_signal],
    [cutoff_normal_signal],
    [cutoff_normal_signal],
]

for i in range(len(reverse_corr_samples)):
    reverse_corr_sample = reverse_corr_samples[i]
    for reverse_corr_signal in reverse_corr_signals[i]:
        reverse_corr_sample.do_reverse_correlation(reverse_corr_signal)
        reverse_corr_sample.display_reverse_corr(reverse_corr_signal)


In [ ]:
#TODO: netrinti visos charakteristikos koreliacijos paliekant impulse,
# bet padidinti konrtarsta tarp peako ir salutiniu dedeamuju. Reiktu naudoti idealu signala.
# APriboti su trikampio, gausiniu filtru tuos sonus. I prieki tarpa ideti, kad paziureti ar ten nieko nera idomauis
#TODO: du variantai, vienas kuris ant duomenu neislenda auksciau preambules, kitas kuris neturi peako pries preambule